# Load Data

In [1]:
import preprocess
import imp
imp.reload(preprocess)

data = preprocess.preprocess_pipeline()

2025-08-28 15:05:11.927823: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756382711.941500 2536057 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756382711.945921 2536057 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756382711.957598 2536057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756382711.957619 2536057 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756382711.957621 2536057 computation_placer.cc:177] computation placer alr

1. Include only first admissions: N=32513
2. Exclusion by ages: N=25548
3. Include only patients who admitted for at least 24 hours: N=25168
4. Exclude patients who died within 54-hours of admission: N=24556


/home/bnet/kupershmidt/HML/preprocess.py:209: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pivot = pd.pivot_table(merged, index=['subject_id', 'hadm_id', pd.Grouper(key='charttime', freq=PREDICT_FREQ)],
/home/bnet/kupershmidt/HML/preprocess.py:269: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=True)
/home/bnet/kupershmidt/HML/preprocess.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val.drop(columns=["mort_30day","prolonged_stay","readmission_30day"], axis=1, inplace=

Pooled embeddings shape: 24529
Number of subjects: 24529


# Datasets creation

In [2]:
data["prescriptions_train"].shape[0] == data["padded_tensor_train"].shape[0]

True

In [3]:
hyperparameters = {'hidden_dim': 128, 'batch_size': 128, 'n1_gat_layers': 1, 'n2_gru_layers': 2, 'num_heads': 4, 'dropout': 0.04033931265087129, 'learning_rate': 0.00015574186652855083, 'pos_lambda': 0.5756532880616873, 'bios_hidden_dim': 32, 'pres_hidden_dim': 64, 'k': 5, 'num_clusters': 182}

In [7]:
import Dataset
import imp
imp.reload(Dataset)
import torch
from torch.utils.data import DataLoader
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
k = hyperparameters['k']

train_labels = torch.tensor(data["y_train"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
val_labels = torch.tensor(data["y_val"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)
test_labels = torch.tensor(data["y_test"][['mort_30day', 'prolonged_stay', 'readmission_30day']].values, dtype=torch.float32).to(DEVICE)


batch_size = hyperparameters['batch_size']
datasets = {x: Dataset.PatientDataset(d, y, core=data["padded_tensor_core"], padding_mask=m, padding_mask_core=data["padding_mask_core"], k=k ,notes=n, bios=b, prescriptions=p) for x, d, y, m, n, b, p in
        zip(['train', 'val', 'test'], [data["padded_tensor_train"], data["padded_tensor_val"], data["padded_tensor_test"]],
            [train_labels, val_labels, test_labels],
            [data["padding_mask_train"], data["padding_mask_val"], data["padding_mask_test"]],
            [torch.stack(data["notes_df_train"].embeddings.values.tolist()),
             torch.stack(data["notes_df_val"].embeddings.values.tolist()),
             torch.stack(data["notes_df_test"].embeddings.values.tolist())],
             [torch.tensor(data["bio_train"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_val"].values >= 1, dtype=torch.float32).to(DEVICE),
              torch.tensor(data["bio_test"].values >= 1, dtype=torch.float32).to(DEVICE)],
              [torch.tensor(data["prescriptions_train"].values, dtype=torch.float32).to(DEVICE), 
              torch.tensor(data["prescriptions_val"].values, dtype=torch.float32).to(DEVICE), 
              torch.tensor(data["prescriptions_test"].values, dtype=torch.float32).to(DEVICE)])}
dataloaders = {x: DataLoader(datasets[x], batch_size=batch_size, shuffle=True) for x in ['train', 'val', 'test']}

# Model Training

In [11]:
import Model 
import imp
imp.reload(Model)
model = Model.GraphGRUMortalityModel(input_dim=data["padded_tensor_train"].shape[2], hidden_dim=hyperparameters['hidden_dim'], 
                                     n1_gat_layers=1, n2_gru_layers=1, 
                                     X_core=data["padded_tensor_core"], num_of_bios=data["bio_train"].shape[1],
                                     num_prescriptions=data["prescriptions_train"].shape[-1], 
                                     bios_hidden_dim=hyperparameters['bios_hidden_dim'], pres_hidden_dim=hyperparameters['pres_hidden_dim'], 
                                     core_padding_mask=data["padding_mask_core"], num_heads=hyperparameters['num_heads'], 
                                     dropout=hyperparameters['dropout'], seq_len=data["padded_tensor_train"].shape[1], k=k, gnn_flag=True).to(DEVICE)
torch.manual_seed(42)
model.train_all(dataloaders, datasets, epochs=15, learning_rate=hyperparameters['learning_rate'])
print("Training completed. Validating on test set...")

model.validate(dataloaders['test'], datasets['test'])
#print(f'Test Accuracy in model: {acc:.4f}, AUC: {auc:.4f}, AP: {pr:.4f}')

/home/bnet/kupershmidt/anaconda3/envs/spider/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.04033931265087129 and num_layers=1
  warnings.warn(


Pos weight 0: 7.5538
Pos weight 1: 1.1253
Pos weight 2: 23.8500
Starting epoch 1/15
100%|██████████| 152/152 [01:31<00:00,  1.66it/s]
Epoch 1/15, Loss: 3.1087
100%|██████████| 20/20 [00:11<00:00,  1.74it/s]
Val Mortality - AUC: 0.7934 | AP: 0.3412
Val Prolonged LOS - AUC: 0.7398 | AP: 0.7129
Val Readmission - AUC: 0.6141 | AP: 0.0579
Best model updated
Starting epoch 2/15
100%|██████████| 152/152 [01:31<00:00,  1.66it/s]
Epoch 2/15, Loss: 2.8927
100%|██████████| 20/20 [00:11<00:00,  1.72it/s]
Val Mortality - AUC: 0.8077 | AP: 0.3566
Val Prolonged LOS - AUC: 0.7741 | AP: 0.7475
Val Readmission - AUC: 0.6482 | AP: 0.0696
Best model updated
Starting epoch 3/15
100%|██████████| 152/152 [01:31<00:00,  1.66it/s]
Epoch 3/15, Loss: 2.8105
100%|██████████| 20/20 [00:11<00:00,  1.74it/s]
Val Mortality - AUC: 0.8196 | AP: 0.3794
Val Prolonged LOS - AUC: 0.7839 | AP: 0.7537
Val Readmission - AUC: 0.6587 | AP: 0.0676
Best model updated
Starting epoch 4/15
100%|██████████| 152/152 [01:30<00:00,  1.6

(0.8697208361082424,
 0.5294962372610046,
 0.8098144982673646,
 0.7859860250898981,
 0.6328291616170507,
 0.09066275128605009)

In [ ]:
res = model.validate(dataloaders['test'], datasets['test'])

print(f"Test Mortality: AUC-ROC: {res[0]:.2f}  AP: {res[1]:.2f}")
print(f"Test Prolonged Stay: AUC-ROC: {res[2]:.2f}  AP: {res[3]:.2f}")
print(f"Test Readmission: AUC-ROC: {res[4]:.2f}  AP: {res[5]:.2f}")

Test Mortality: AUC-ROC: 0.87  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.83  AP: 0.81
Test Readmission: AUC-ROC: 0.63  AP: 0.08


<!-- 100%|██████████| 10/10 [00:07<00:00,  1.38it/s]
Test Mortality: AUC-ROC: 0.85  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.82  AP: 0.80
Test Readmission: AUC-ROC: 0.62  AP: 0.08 -->

<!-- Test Mortality: AUC-ROC: 0.87  AP: 0.52
Test Prolonged Stay: AUC-ROC: 0.83  AP: 0.81
Test Readmission: AUC-ROC: 0.62  AP: 0.08 -->

In [ ]:
model.validate(dataloaders['test'], datasets['test'])


100%|██████████| 10/10 [00:07<00:00,  1.39it/s]


(0.862270225393303,
 0.5374146532039636,
 0.8114620700121142,
 0.7750227972338779,
 0.5760484176849882,
 0.05048034000235363)